In [1]:
import pyvisa as visa
import sounddevice as sd
import soundfile as sf
import numpy as np
from time import sleep
import wave as wav
from contextlib import closing
import struct

import os

In [9]:
t_freq = 7    # Time per frequency
t_pause = 3  # Pause between two frequencies
#n_freq = 20   # Number of different frequencies
f_min = 500    # Minimum frequency
f_max = 20e3  # Maximum frequency

f_c = 40e3    # Carrier frequency

fs = 300e3    # Samplingrate
vf = [50, 100, 200, 400, 600, 800, 1000, 1200, 1500, 2000, 3000, 4000, 5000, 8000, 10000, 15000, 20000] #np.linspace(f_min, f_max, n_freq, dtype=np.int64)

# Configure Waveform Generator

In [10]:
def enableAWG(inst, channel: int = 1):
  inst.write(f"OUTP{channel} ON")

def disableAWG(inst, channel: int = 1):
  inst.write(f"OUTP{channel} OFF")

def configureSignal(inst, f: int, amplitude: float, offset:float, function_type: str = "SIN", channel: int = 1):
  base = f"SOUR{channel}"

  inst.write(f"OUTP{channel}:LOAD INF")
  inst.write(f"{base}:FUNC {function_type}; FREQ {int(f)}; VOLT {amplitude}")
  inst.write(f"{base}:VOLT:OFFS {offset}")

def enableModulation(inst, type: str, m: float, f: int, function_type: str = "SIN", channel: int = 1):
  base = f"SOUR{channel}:{type}"
  inst.write(f"{base}:STAT ON; INT:FREQ {f}")

  if type == "AM":
    inst.write(f"{base}:DEPT {int(m * 100)}")
  elif type == "FM":
    inst.write(f"{base}:DEV {int(m)}")

def disableModulation(inst, type: str, channel: int = 1):
  inst.write(f"SOUR{channel}:{type}:STAT OFF")

In [4]:
rm = visa.ResourceManager("@py")
resources = rm.list_resources()
resources

('USB0::10893::36097::CN61150030::0::INSTR',)

In [5]:
wav_gen_id = resources[0]
wav_gen = rm.open_resource(wav_gen_id)

## Set Carrier Signal

In [6]:
configureSignal(wav_gen, f_c, 4, 2.5)

# Measure

In [28]:
modulation_types=["AM", "FM"]
modulation_depth=[0.8, 5e3]

angle=80

t_record = (t_freq + t_pause + 0.5)
t_silence = 10
t_before_start = 20

channels = [3, 4]
fs_record = 44100

(2 * t_record * len(vf) + t_silence + t_before_start)/60

6.45

In [29]:
from datetime import datetime

disableAWG(wav_gen)
rec = np.zeros(shape=(int(fs_record * t_record), len(channels)), dtype=np.float32)

sleep(t_before_start) # Leave the room

sd.rec(out=rec, mapping=channels)
sleep(t_silence)
sd.stop()
sf.write(f"recordsV2/noise_{datetime.now().isoformat()}.wav",
           rec[:, 0], fs_record, "PCM_24")

for modulation_idx, modulation_type in enumerate(modulation_types):
  for f in vf:
    basepath = f"recordsV2/{modulation_type}/{round(int(f)*1e-3, 2)}kHz"
    sd.rec(out=rec, mapping=channels)
    os.makedirs(basepath, exist_ok=True)
    sleep(t_pause)
    enableModulation(wav_gen, modulation_type, modulation_depth[modulation_idx], f)
    enableAWG(wav_gen)
    sleep(t_freq)
    disableAWG(wav_gen)
    sd.stop()

    sf.write(f"{basepath}/meas_{angle}deg.wav",
            rec[:, 0], fs_record, "PCM_24")
    sf.write(f"{basepath}/meas_{angle + 180}deg.wav",
            rec[:, 1], fs_record, "PCM_24")


## Close Device

In [17]:
rm.close()

In [8]:
t_record = 2
channels = [3, 4]

rec = np.zeros(shape=(fs_record * t_record, len(channels)), dtype=np.float32)
enableModulation(wav_gen, "FM", 5e3, 1e3)
sd.rec(out=rec, mapping=channels)
enableAWG(wav_gen)
sd.wait()
disableAWG(wav_gen)


In [94]:
sd.play(rec[:,1])
sd.wait()